# Python装饰器

装饰器本质上是一个 Python 函数或类，它可以让其他函数或类在不需要做任何代码修改的前提下增加额外功能，装饰器的返回值也是一个函数/类对象。它经常用于有切面需求的场景，比如：插入日志、性能测试、事务处理、缓存、权限校验等场景，装饰器是解决这类问题的绝佳设计。有了装饰器，我们就可以抽离出大量与函数功能本身无关的雷同代码到装饰器中并继续重用。概括的讲，装饰器的作用就是为已经存在的对象添加额外的功能。

## 参考
* [Python装饰器学习（九步入门）](http://www.cnblogs.com/rhcad/archive/2011/12/21/2295507.html)
* [Python 装饰器 - 伯乐在线](http://python.jobbole.com/82344/)
* [理解 Python 装饰器看这一篇就够了](https://foofish.net/python-decorator.html)
* [Python中的各种装饰器详解](http://www.jb51.net/article/63892.htm)

## 函数式装饰器
以对函数实现函数增加计数执行时间功能为例子。

### 装饰函数

错误的示范：

In [7]:
import time
def deco(func):
    print("%s is start..." % func.__name__)
    start_time = time.time()
    func()
    end_time = time.time()
    delta = (end_time - start_time) * 1000
    print("%s is ended with %fms" % (func.__name__, delta))
    return func

@deco
def myfunc():
    time.sleep(0.6)

myfunc()
myfunc()

myfunc is start...
myfunc is ended with 609.466314ms


装饰的内容只被调用了一次。由此可以判断：
1. 装饰函数（即上面的deco），只在第一次调用被装饰函数（myfunc()）时执行一次
2. 由于装饰函数返回的还是func，所以第二次调用的还是func，并没有附加内容。

正确的做法是返回包装之后的函数，而不是被装饰函数本身：

In [9]:
def deco(func):
    def _deco():
        print("%s is start..." % func.__name__)
        start_time = time.time()
        func()
        end_time = time.time()
        delta = (end_time - start_time) * 1000
        print("%s is ended with %fms" % (func.__name__, delta))
        
    return _deco

@deco
def myfunc():
    time.sleep(0.6)

myfunc()
myfunc()

myfunc is start...
myfunc is ended with 609.981060ms
myfunc is start...
myfunc is ended with 604.786396ms


总结：
1. 装饰器本身是一个函数。
2. 以被装饰的函数为输入参数。
3. 返回被包装之后的函数。

上例实际上等价于下面的实现，装饰器用法其实是语法糖：

In [10]:
def deco(func):
    def _deco():
        print("%s is start..." % func.__name__)
        start_time = time.time()
        func()
        end_time = time.time()
        delta = (end_time - start_time) * 1000
        print("%s is ended with %fms" % (func.__name__, delta))
        
    return _deco

# 注意这时没有用@
def myfunc():
    time.sleep(0.6)

deco(myfunc)()
deco(myfunc)()

myfunc is start...
myfunc is ended with 613.054991ms
myfunc is start...
myfunc is ended with 608.532429ms


### 装饰有参数的函数
有时被装饰函数是有参数的，这时应该在包装函数（即_deco）中接收相应的参数，因为包装函数实际上替代了被装饰的函数。

In [11]:
def deco(func):
    def _deco(a, b):
        print("%s is start..." % func.__name__)
        start_time = time.time()
        result = func(a, b)
        end_time = time.time()
        delta = (end_time - start_time) * 1000
        print("%s is ended with %fms" % (func.__name__, delta))
        return result
        
    return _deco

@deco
def myfunc(a, b):
    time.sleep(0.6)
    return a + b

print(myfunc(1, 3))
print(myfunc(6, 2))

myfunc is start...
myfunc is ended with 600.679636ms
4
myfunc is start...
myfunc is ended with 605.173111ms
8


实际上可以当作是参数不定情况，统一使用解包进行接收：

In [12]:
def deco(func):
    def _deco(*args, **kwargs):
        print("%s is start..." % func.__name__)
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        delta = (end_time - start_time) * 1000
        print("%s is ended with %fms" % (func.__name__, delta))
        return result
        
    return _deco

@deco
def myfunc(a, b):
    time.sleep(0.6)
    return a + b

print(myfunc(1, 3))
print(myfunc(6, 2))

myfunc is start...
myfunc is ended with 611.173391ms
4
myfunc is start...
myfunc is ended with 601.498842ms
8


等同于调用了`deco(func)(1, 3)`和`deco(func)(6, 2)`

### 装饰器带参数
有时装饰器可以传入参数进行更细致的控制。比如通过参数控制是否进行计算执行时间：

In [17]:
def deco(time_it=True):
    if time_it:
        def _deco(func):
            def __deco(*args, **kwargs):
                print("%s is start..." % func.__name__)
                start_time = time.time()
                result = func(*args, **kwargs)
                end_time = time.time()
                delta = (end_time - start_time) * 1000
                print("%s is ended with %fms" % (func.__name__, delta))
                return result
            return __deco
        return _deco
    else:
        def _deco(func):
            def __deco(*args, **kwargs):
                return func(*args, **kwargs)
            return __deco
        return _deco

@deco(True)
def myfunc(a, b):
    time.sleep(0.6)
    return a + b

@deco(False)
def myfunc2(a, b):
    time.sleep(0.6)
    return a + b

print(myfunc(1, 3))
print(myfunc2(6, 2))

myfunc is start...
myfunc is ended with 602.722168ms
4
8


此时只需在不带参数的装饰器函数外面再包一层函数用于接收参数即可。实际上等同于`deco(time_it=True)(myfunc)(1, 3)`。·

### 装饰类
装饰器可以用于类上面，对类的属性或行为进行修改。

In [23]:
def deco(cls):
    def _deco():
        c = cls()
        c.a = 10
        return c
    return _deco

@deco
class C:
    pass

c = C()
c.a

10

实际和装饰函数没有什么两样。
如果类构造器有参数，则：

In [25]:
def deco(cls):
    def _deco(*args, **kwargs):
        c = cls(*args, **kwargs)
        c.a = 10
        return c
    return _deco

@deco
class C:
    def __init__(self, value):
        self.value = value

c = C(100)
print(c.a)
print(c.value)

10
100


装饰器带参数：

In [28]:
def deco(printValue=True):
    def _deco(cls):
        def __deco(*args, **kwargs):
            c = cls(*args, **kwargs)
            c.a = 10
            if printValue:
                print('Value:', c.value)
            return c
        return __deco
    return _deco

@deco
class C:
    def __init__(self, value):
        self.value = value

c = C(100)